# Ejercicio 5
## En este caso se hace una extension de los dedos de la mano por ello no se utilizan los angulos del codo

In [25]:
import pandas as pd


with open('/content/drive/MyDrive/pract3Landmrks/labelsejer5.json', 'r') as f:
    data = json.load(f)


X = []
y = []
for video_name, frames in data.items():
    features = extract_video_features(frames)
    X.append(features)
    y.append(frames[0]['label'])  #

df = pd.DataFrame(X)
df['label'] = y

In [26]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

X = df.drop('label', axis=1)
y = df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)


models = {
    'SVM': SVC(probability=True),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': GradientBoostingClassifier()
}


param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        full       1.00      1.00      1.00         2
        none       0.00      0.00      0.00         2
     partial       0.50      1.00      0.67         2

    accuracy                           0.67         6
   macro avg       0.50      0.67      0.56         6
weighted avg       0.50      0.67      0.56         6



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Con un modelo de SVM no se lorgra una clasifficación aceptable, ademas no se tienen caracteristicas suficientes para none

In [27]:
import json
import numpy as np
import os

def load_landmark_data(file_path):

    if not os.path.isfile(file_path):
        raise ValueError(f"La ruta {file_path} no es un archivo válido.")


    with open(file_path, "r") as f:
        data = json.load(f)

    X, y = [], []

    max_landmarks = 0

    for video_name, frames in data.items():
        for frame_data in frames:
            landmarks = []


            pose = frame_data.get("pose", {})
            hands = frame_data.get("hands", [])


            for key in ["left_arm", "right_arm"]:
                if key in pose:
                    for coords in pose[key]:
                        landmarks.extend(coords)


            for hand_points in hands:
                for coords in hand_points:
                    landmarks.extend(coords)


            max_landmarks = max(max_landmarks, len(landmarks))


            X.append(landmarks)
            y.append(frame_data["label"])


    for i in range(len(X)):
        while len(X[i]) < max_landmarks:
            X[i].append(0.0)

    return np.array(X), np.array(y)


file_path = "/content/drive/MyDrive/pract3Landmrks/labelsejer5.json"


X, y = load_landmark_data(file_path)
X = X / np.max(X)

print(f"Datos cargados: {X.shape}, Etiquetas: {len(y)}")

Datos cargados: (7479, 207), Etiquetas: 7479


In [28]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


model = SVC(kernel="linear")
model.fit(X_train, y_train_encoded)

SVC(kernel='linear')

In [29]:
from sklearn.metrics import accuracy_score, classification_report


y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Precisión del modelo: {accuracy:.3f}")


print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))


Precisión del modelo: 0.667
              precision    recall  f1-score   support

        full       1.00      1.00      1.00         2
        none       0.00      0.00      0.00         2
     partial       0.50      1.00      0.67         2

    accuracy                           0.67         6
   macro avg       0.50      0.67      0.56         6
weighted avg       0.50      0.67      0.56         6



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Se aplica un modelo de MLP Classifiers pues se había logrado un buen desempeño para otros ejercicios

In [30]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
model_mlp.fit(X_train_scaled, y_train_encoded)

y_pred_mlp = model_mlp.predict(X_test_scaled)
print(classification_report(y_test_encoded, y_pred_mlp, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

        full       1.00      0.50      0.67         2
        none       0.00      0.00      0.00         2
     partial       0.25      0.50      0.33         2

    accuracy                           0.33         6
   macro avg       0.42      0.33      0.33         6
weighted avg       0.42      0.33      0.33         6



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## No se logra un resultado favorable parala clasificación del ejercicio